We begin by loading some of the necessary libraries!

In [2]:
import numpy as np
import pandas as pd

import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

print('Libraries imported')


Libraries imported


To scrape the data, I saved the web page as a pdf and then copied and pasted it into excel to create a .csv file that could be inserted into the code to create the data frame. 

In [11]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M5N,Central Toronto,Roselawn
2,M4P,Central Toronto,Davisville North
3,M5P,Central Toronto,Forest Hill North & West
4,M4R,Central Toronto,North Toronto West


In [3]:
postal_df.tail()

,Postal Code,Borough,Neighborhood
98,M6C,York,Humewood-Cedarvale
99,M6E,York,Caledonia-Fairbanks
100,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
101,M6N,York,"Runnymede, The Junction North"
102,M9N,York,Weston


I have already eliminated the 'not assigned' entries from the csv before inserting it in the code